# AMS Project 553
### Benjamin Nicholson & Nick Christophedes

As part of AMS 553 at Stony Brook University the students have been tasked with a final project that looks to utilize skills learned from the class to create an advanced queueing simulation. As both Ben and Nick are Quantitative Finance students they saw an opportunity to utilize queueing theory to emulate the Log Order Book (LOB)

## Methodology

In [138]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heapq
import simpy

In [139]:
class Distribution:
    """
    Distributions class will be used for random variate generation
    """
    def __init__(self, sampler):
        self.sampler = sampler

    def sample(self):
        return self.sampler()

In [140]:
class Order:
    def __init__(self, investor_id, price, time, side):
        """
        Attrbiutes:
        investor_id: Unique identifier for the investor placing the order
        price: The price at which the order is placed
        time: The timestamp when the order is placed
        side: 'buy' or 'sell' indicating the type of order

        """
        self.investor_id = investor_id
        self.price = price
        self.time = time
        self.side = side

In [144]:
class OrderBook:
    """
    Attributes:
    - p0: initial price of the asset
    - bids: list of buy orders (max heap)
    - asks: list of sell orders (min heap)
    
    Methods:
    - best_bid(): returns the highest bid price
    - best_ask(): returns the lowest ask price
    - midpoint_price(): returns the midpoint price between best bid and best ask
    - add_order(order): adds an order to the order book
    """
    def __init__(self,p0):
        self.p0 = p0
        self.bids = []  # max heap 
        self.asks = []  # min heap
        self.best_bid_list = []
        self.best_ask_list = []

    def best_bid(self):
        current_best_bid = -self.bids[0][0] if self.bids else None # use negative as heapq finds minimum 
        self.best_bid_list.append(current_best_bid)
        return current_best_bid

    def best_ask(self):
        current_best_ask = self.asks[0][0] if self.asks else None
        self.best_ask_list.append(current_best_ask)
        return 

    def midpoint_price(self):
        bid = self.best_bid()
        ask = self.best_ask()
        if bid is None or ask is None: # i need to initialize the order book with a p0
            return self.p0
        return (bid + ask) / 2

    def add_order(self, order):
        if order.side == "buy":
            heapq.heappush(self.bids, (-order.price, order))
        else:
            heapq.heappush(self.asks, (order.price, order))

At each time step the order book is going to update with new method values. This allows us to track how these values change over time

In [145]:
class Investor:
    """
    Attributes:
    - id: Unique identifier for the investor
    - price_dist: Distribution object for price noise
    - arrival_dist: Distribution object for inter-arrival times
    Methods:
    - get_valuation(orderbook): returns the valuation of the investor based on the order book
    - map_price(val, noise): maps the valuation and noise to a price (to be implemented in subclasses)
    - generate_price(orderbook): generates a price based on valuation and noise
    """
    def __init__(self, id, price_dist, arrival_dist):
        self.id = id
        self.price_dist = price_dist
        self.arrival_dist = arrival_dist

    def get_valuation(self, orderbook):
        return orderbook.midpoint_price() # we get the value of an investor to be the mid point between best ask and best bid

    def map_price(self, val, noise):
        raise NotImplementedError

    def generate_price(self, orderbook):
        noise = self.price_dist.sample()
        val = self.get_valuation(orderbook)
        return round(self.map_price(val, noise), 2)
    
class Buyer(Investor):
    def map_price(self, val, noise): # we make sure that the noise is to the left of the midpoint as buyers will make lower than midpoint bids
        return val - noise

    def run(self, env, orderbook):
        while True:
            yield env.timeout(self.arrival_dist.sample())
            price = self.generate_price(orderbook) # we generate price by fetching the value which gets the midpoint of the orderbook between bid and ask
            order = Order(self.id, price, env.now, "buy") # we make the order per interarrival time
            orderbook.add_order(order)


class Seller(Investor):
    def map_price(self, val, noise):
        return val + noise

    def run(self, env, orderbook):
        while True:
            yield env.timeout(self.arrival_dist.sample())
            price = self.generate_price(orderbook)
            order = Order(self.id, price, env.now, "sell")
            orderbook.add_order(order)

In [146]:
env = simpy.Environment()
orderbook = OrderBook(100)

arrival_dist = Distribution(lambda: np.random.exponential(10)) # we want 10 arrivals a second
price_dist = Distribution(lambda: np.random.exponential(2)) # we want the scale to be 2

buyers = [Buyer(i, price_dist, arrival_dist) for i in range(3)]
sellers = [Seller(i+100, price_dist, arrival_dist) for i in range(3)]

for b in buyers:
    env.process(b.run(env, orderbook))
for s in sellers:
    env.process(s.run(env, orderbook))

env.run(until=10)

In [150]:
orderbook.best_ask_list
orderbook.best_bid_list

[None, None, None, None, None, 98.83, 98.83, 98.83]